In [1]:
# Imports
import IPython
import IPython.display
import pandas as pd
from numpy import mean
from numpy import std
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
import glob
import tensorflow as tf
import math
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

np.random.seed(37151)
tf.random.set_seed(37151)

In [2]:
time = 2.0
steps = int(time * 200)

In [3]:
class LSTMClassification():
    def __init__(self):
         self.df = pd.read_csv('../master_df.csv')     
    
    # model_output_size = numero de categorias a determinar
    # mag_offset = valor magnitudeminimo a categorizar
    def prep_features(self, model_output_size=3, mag_offset=4, timedelta = 400):
        self.timedelta = timedelta
        features = []
        labels = []
        for index, row in self.df.iterrows():
            try:
                records = row['list_records'].replace("'", '')
                records = records.replace('[', '')
                records = records.replace(']', '')                                  
                list_records = records.split(', ')
                magnitude = row['Mag']
                for record in list_records:
                    df_ts = pd.read_csv("../Renamed_Records/" + record, index_col=None, header=0, dtype=np.float32)
                    features.append(df_ts.iloc[:timedelta].to_numpy(dtype='float'))
                    labels.append(np.eye(model_output_size)[magnitude-mag_offset])
            except:
                continue
        self.features=np.array(features)
        self.labels=np.array(labels)
    
    def split_dataset(self, split = 0.8):
        self.trainX, self.testX, self.trainY, self.testY = train_test_split(self.features, self.labels, test_size = split, shuffle = True)
        print(self.trainX)
        print(self.trainY)
    
    def create_model(self, height=-1, width=2, model_output_size=3):
        model = Sequential()
        if height == -1:
            height = self.timedelta

        # input shape = num timesteps, num features(temp, acc.)
        model.add(LSTM(100, input_shape = (height, width)))
        model.add(Dropout(0.5))
        model.add(Dense(100, activation = 'relu'))
        model.add(Dense(model_output_size, activation = 'softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        # Printing the models summary
        #model.summary()
        return model
    
    def evaluate_model(self, model, trainX, trainY, testX, testY, epochs=100, batch_size=3, verbose=0):
        history = model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=verbose)
        loss, accuracy = model.evaluate(testX, testY, batch_size, verbose=verbose)
        return accuracy
    
    def predict_model(self, model, data):
        features = data['Values'].to_numpy().astype(float)
        predictions = model.predict(features)
        return predictions

In [4]:
# 2 segundos

classifier = LSTMClassification()
classifier.prep_features(timedelta = 2*200)
print(classifier.features.shape)
print(classifier.labels.shape)

classifier.split_dataset()
# height = numero de timestamps, width (time, acc.)
LSTM_magnitude_model = classifier.create_model()

# struct: (Number or files, number of steps, number of columns)
# (70, 400, 2)
classifier.evaluate_model(LSTM_magnitude_model, classifier.trainX,  classifier.trainY,  classifier.testX,  classifier.testY, verbose=1)

(58, 400, 2)
(58, 3)
[[[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  1.22947185e-05]
  [ 9.99999978e-03  2.46343043e-05]
  ...
  [ 1.98500001e+00  1.02228182e-03]
  [ 1.99000001e+00 -2.86151742e-04]
  [ 1.99500000e+00 -1.52845471e-03]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -7.74984343e-08]
  [ 9.99999978e-03 -1.54996869e-07]
  ...
  [ 1.98500001e+00  6.30592513e-06]
  [ 1.99000001e+00  6.32224055e-06]
  [ 1.99500000e+00  6.33753643e-06]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  7.50028819e-04]
  [ 9.99999978e-03  1.49745739e-03]
  ...
  [ 1.98500001e+00 -3.22904401e-02]
  [ 1.99000001e+00 -3.09988074e-02]
  [ 1.99500000e+00 -2.86463331e-02]]

 ...

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -3.02233693e-05]
  [ 9.99999978e-03 -6.04548950e-05]
  ...
  [ 1.98500001e+00  4.82581730e-04]
  [ 1.99000001e+00 -1.40420836e-03]
  [ 1.99500000e+00 -2.86069862e-03]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  1.60005715e-04]
  [ 9

0.8936170339584351

In [5]:
# 4 segundos

classifier = LSTMClassification()
classifier.prep_features(timedelta = 4*200)
print(classifier.features.shape)
print(classifier.labels.shape)

classifier.split_dataset()
# height = numero de timestamps, width (time, acc.)
LSTM_magnitude_model = classifier.create_model()

# struct: (Number or files, number of steps, number of columns)
# (70, 400, 2)
classifier.evaluate_model(LSTM_magnitude_model, classifier.trainX,  classifier.trainY,  classifier.testX,  classifier.testY, verbose=1)

(58, 800, 2)
(58, 3)
[[[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  5.87638002e-04]
  [ 9.99999978e-03  1.17483851e-03]
  ...
  [ 3.98499990e+00 -1.73398759e-02]
  [ 3.99000001e+00 -1.74183883e-02]
  [ 3.99499989e+00 -1.71211399e-02]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -3.46703501e-07]
  [ 9.99999978e-03 -6.93407003e-07]
  ...
  [ 3.98499990e+00 -1.83614175e-04]
  [ 3.99000001e+00 -1.28043728e-04]
  [ 3.99499989e+00 -3.00438987e-05]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -3.87492172e-08]
  [ 9.99999978e-03 -7.74984343e-08]
  ...
  [ 3.98499990e+00 -3.57920385e-07]
  [ 3.99000001e+00 -1.79470049e-07]
  [ 3.99499989e+00  1.01971622e-08]]

 ...

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -8.53161910e-04]
  [ 9.99999978e-03 -1.70531217e-03]
  ...
  [ 3.98499990e+00 -9.17477533e-03]
  [ 3.99000001e+00 -8.04918259e-03]
  [ 3.99499989e+00 -6.91993069e-03]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -6.73012721e-07]
  [ 9

0.8723404407501221

In [6]:
# 5 segundos

classifier = LSTMClassification()
classifier.prep_features(timedelta = 5*200)
print(classifier.features.shape)
print(classifier.labels.shape)

classifier.split_dataset()
# height = numero de timestamps, width (time, acc.)
LSTM_magnitude_model = classifier.create_model()

# struct: (Number or files, number of steps, number of columns)
# (70, 400, 2)
classifier.evaluate_model(LSTM_magnitude_model, classifier.trainX,  classifier.trainY,  classifier.testX,  classifier.testY, verbose=1)

(58, 1000, 2)
(58, 3)
[[[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  3.08056269e-06]
  [ 9.99999978e-03  6.15806630e-06]
  ...
  [ 4.98500013e+00  2.22247152e-04]
  [ 4.98999977e+00  2.23694122e-04]
  [ 4.99499989e+00  2.25128868e-04]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  3.83950683e-04]
  [ 9.99999978e-03  7.66638957e-04]
  ...
  [ 4.98500013e+00 -8.11374839e-03]
  [ 4.98999977e+00 -7.11021293e-03]
  [ 4.99499989e+00 -5.89875132e-03]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  1.46992088e-05]
  [ 9.99999978e-03  2.94494039e-05]
  ...
  [ 4.98500013e+00  3.30737606e-03]
  [ 4.98999977e+00  4.96709999e-03]
  [ 4.99499989e+00  6.30662451e-03]]

 ...

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  3.40585217e-07]
  [ 9.99999978e-03  6.80150720e-07]
  ...
  [ 4.98500013e+00 -1.84937773e-04]
  [ 4.98999977e+00 -2.81526300e-04]
  [ 4.99499989e+00 -2.75612983e-04]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -1.57263697e-04]
  [ 

0.8297872543334961

In [7]:
# 8 segundos

classifier = LSTMClassification()
classifier.prep_features(timedelta = 8*200)
print(classifier.features.shape)
print(classifier.labels.shape)

classifier.split_dataset()
# height = numero de timestamps, width (time, acc.)
LSTM_magnitude_model = classifier.create_model()

# struct: (Number or files, number of steps, number of columns)
# (70, 400, 2)
classifier.evaluate_model(LSTM_magnitude_model, classifier.trainX,  classifier.trainY,  classifier.testX,  classifier.testY, verbose=1)

(58, 1600, 2)
(58, 3)
[[[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  3.83950683e-04]
  [ 9.99999978e-03  7.66638957e-04]
  ...
  [ 7.98500013e+00 -3.77487321e-03]
  [ 7.98999977e+00 -4.05027112e-03]
  [ 7.99499989e+00 -4.33966331e-03]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -3.46703501e-07]
  [ 9.99999978e-03 -6.93407003e-07]
  ...
  [ 7.98500013e+00  4.90442726e-05]
  [ 7.98999977e+00 -5.29151148e-05]
  [ 7.99499989e+00 -1.54482928e-04]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  1.10261914e-04]
  [ 9.99999978e-03  2.20470800e-04]
  ...
  [ 7.98500013e+00 -9.83684510e-03]
  [ 7.98999977e+00 -1.11516202e-02]
  [ 7.99499989e+00 -1.15458583e-02]]

 ...

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -2.51971869e-06]
  [ 9.99999978e-03 -5.03127967e-06]
  ...
  [ 7.98500013e+00  4.38736901e-02]
  [ 7.98999977e+00  6.63262308e-02]
  [ 7.99499989e+00  9.74326432e-02]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -1.57263697e-04]
  [ 

0.8936170339584351